In [ ]:
import psycopg2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
conn = psycopg2.connect(
    host='localhost',
    database='db',
    user='postgres',
    password='postgres'
)

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute("SELECT version()")

In [ ]:
db_version = cur.fetchone()

In [ ]:
print(db_version)

In [ ]:
all_stock_code_sql = """
    SELECT stock.stock_code
    FROM stock
    ORDER BY stock.stock_code;
"""

In [ ]:
cur.execute(all_stock_code_sql)

In [ ]:
res = cur.fetchall()

In [ ]:
print(res)
stock_codes = [x[0] for x in res]
print(stock_codes)

In [ ]:
select_history_sql = """
    SELECT history.date, history.close, history.stock_code
    FROM history
    ORDER BY history.stock_code, history.date;
"""
# cur.execute(select_history_sql)
# res = cur.fetchall()
# print(res)

In [ ]:
# for x in res[0]:
#     print(x, type(x))

In [ ]:
# for x in res:
#     print(x)

In [ ]:
def plot_series(time, series, start=0, end=None, format="-"):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

In [ ]:
p_sql = """
    SELECT * FROM get_stock_history('%s');
""" % ("0050")

In [ ]:
# cur.execute(p_sql)
# res = cur.fetchall()
# print(res)

In [ ]:
df = pd.read_sql(p_sql, con=conn)

In [ ]:
df

In [ ]:
series = df['close'].values
time = df['date'].values

In [ ]:
plot_series(time, series)

In [ ]:
window_size = 20
batch_size = 32
split_time = 100
shuffle_buffer = 1000

def windowed_dataset(series, window_size=20, batch_size=32, shuffle_buffer=1000):
    series = tf.expand_dims(series, axis=-1)
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

def model_predict(model, series, window_size=20):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_val = series[split_time:]

In [ ]:
train_dataset = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size)
print(type(dataset))
# for window in dataset:
#     x, y = window
#     print(x.numpy(), y.numpy())

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=60,
              split_time        kernel_size=5,
                           strides=1,
                           padding="causal",
                           activation="relu",
                           input_shape=[None, 1]),
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(), 
              optimizer=optimizer,
              metrics=["mae"])
model.summary()
history = model.fit(train_dataset, epochs=500)

In [ ]:
forecast = model_predict(model, series[..., np.newaxis], window_size)
forecast = forecast[split_time - window_size:-1, -1, 0]
# forecast = forecast[]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_val)
plot_series(time_valid, forecast)

In [ ]:
# plt.semilogx(history.history["lr"], history.history["loss"])
# plt.axis([1e-8, 1e-4, 0, 60])

In [ ]:
d1 = tf.data.Dataset.range(10, 20).window(6, shift=1, drop_remainder=True)
d2 = tf.data.Dataset.range(60, 70).window(6, shift=1, drop_remainder=True)

In [ ]:
concat_d = d1.stack(d2)
for w in concat_d:
    for v in w:
        print(v.numpy(), end=' ')
    print()

In [ ]:
conn.close()